# Perimeter Detection
Detects if a person crosses a perimeter area in the camera. Well useful to detect motion in a specified area.

Using `ultralytics`

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Javascript, Image as IPImage
from google.colab.output import eval_js
from google.colab import output

from base64 import b64decode
import cv2
import numpy

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
def js_to_image(js_reply):
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = numpy.frombuffer(image_bytes, dtype=numpy.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
perimeter_top_left = (100, 100)
perimeter_bottom_right = (500, 500)

def is_inside_perimeter(x, y, perimeter_top_left, perimeter_bottom_right):
    return (perimeter_top_left[0] <= x <= perimeter_bottom_right[0] and
            perimeter_top_left[1] <= y <= perimeter_bottom_right[1])


In [ ]:
from PIL import Image

def display_image(image):
    _, encoded_image = cv2.imencode('.png', image)
    image_bytes = encoded_image.tobytes()
    display(IPImage(data=image_bytes))


def predict(file):
    res = model(file, device="0", stream=True)

    cv2.rectangle(file, perimeter_top_left, perimeter_bottom_right, (0, 255, 0), 2)

    for result in res:
        print(result)
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])

            if cls == 0:  # Class 0 is 'person' for YOLO
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2

                # Check if the person is inside the perimeter
                if is_inside_perimeter(center_x, center_y, perimeter_top_left, perimeter_bottom_right):
                    cv2.putText(file, "ALERT!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.rectangle(file, (x1, y1), (x2, y2), (0, 0, 255), 2)
                else:
                    cv2.rectangle(file, (x1, y1), (x2, y2), (255, 0, 0), 2)

    display_image(file)

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }

    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js)

In [ ]:

def video_frame(bbox):
  data = eval_js('stream_frame("{}")'.format(bbox))
  return data


video_stream()

bbox = ''
count = 0
while True:
    js_reply = video_frame(bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    predict(img)

